In [1]:
from caffe.proto import caffe_pb2

In [2]:
from os.path import join

In [13]:
def make_solver(net_path, 
                train_file_name, 
                test_net_path=None, 
                base_lr=0.0001,
                tr_batch_size=4, 
                tr_num_examples=1000,
                test_batch_size=16, 
                test_num_examples=500,
                print_every_iteration=1, max_epochs=5000):
    
    
    tr_iters_per_epoch = int(tr_num_examples / tr_batch_size)
    test_iters_per_epoch = int(test_num_examples / test_batch_size)
    s = caffe_pb2.SolverParameter()

    # Specify locations of the train and (maybe) test networks.
    s.train_net = join(net_path, train_file_name)
    if test_net_path is not None:
        s.test_net.append(test_net_path)
        s.test_interval = tr_iters_per_epoch  # Test after every 1000 training iterations.
        s.test_iter.append(0) # Test on 100 batches each time we test.

    # The number of iterations over which to average the gradient.
    # Effectively boosts the training batch size by the given factor, without
    # affecting memory utilization.
    s.iter_size = 1
    
    s.max_iter = max_epochs * tr_iters_per_epoch     # # of times to update the net (training iterations)
    
    # Solve using the stochastic gradient descent (SGD) algorithm.
    # Other choices include 'Adam' and 'RMSProp'.
    s.type = 'SGD'

    # Set the initial learning rate for SGD.
    s.base_lr = base_lr

    # Set `lr_policy` to define how the learning rate changes during training.
    # Here, we 'step' the learning rate by multiplying it by a factor `gamma`
    # every `stepsize` iterations.
    s.lr_policy = 'step'
    s.gamma = 1.0
    s.stepsize = 20000

    # Set other SGD hyperparameters. Setting a non-zero `momentum` takes a
    # weighted average of the current gradient and previous gradients to make
    # learning more stable. L2 weight decay regularizes learning, to help prevent
    # the model from overfitting.
    s.momentum = 0.9
    s.weight_decay = 5e-4

    # Display the current training loss and accuracy every 1000 iterations.
    s.display = print_every_iteration

    # Snapshots are files used to store networks we've trained.  Here, we'll
    # snapshot every 10K iterations -- ten times during training.
#     s.snapshot = 10000
#     s.snapshot_prefix = net_path + '/models/'
    
    # Train on the GPU.  Using the CPU to train large networks is very slow.
    s.solver_mode = caffe_pb2.SolverParameter.CPU
    
    # Write the solver to a temporary file and return its filename.
    with open(join(net_path, "solver.prototxt"), "w")  as f:
        f.write(str(s))
        return str(s),f.name

In [14]:
#make_solver("/project/projectdirs/dasrepo/gordon_bell/deep_learning/networks/climate/2d_semi_sup", "train.prototxt")

'/project/projectdirs/dasrepo/gordon_bell/deep_learning/networks/climate/2d_semi_sup/solver.prototxt'